In [1]:
# Import libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [30]:
# Load the dataset
df = pd.read_csv("obesity_dataset.csv")
df.head()

,Gender,Height,Age,Meals_Per_Day,Frequency_Physical_Activity,Water_Intake,Vegetable_Intake,Technology_Use,Weight,family_history_with_overweight,High_Caloric_Food,Food_Between_Meals,SMOKE,Monitor_Calories,Alcohol_Intake,Mode_Transportation,Obesity_Level,Obese/Overweight
0,Female,1.62,21.0,3.0,0.0,2.0,2.0,1.0,64.0,yes,no,Sometimes,no,no,no,Public_Transportation,Normal_Weight,No
1,Female,1.52,21.0,3.0,3.0,3.0,3.0,0.0,56.0,yes,no,Sometimes,yes,yes,Sometimes,Public_Transportation,Normal_Weight,No
2,Male,1.80,23.0,3.0,2.0,2.0,2.0,1.0,77.0,yes,no,Sometimes,no,no,Frequently,Public_Transportation,Normal_Weight,No
3,Male,1.80,27.0,3.0,2.0,2.0,3.0,0.0,87.0,no,no,Sometimes,no,no,Frequently,Walking,Overweight_Level_I,Yes
4,Male,1.78,22.0,1.0,0.0,2.0,2.0,0.0,89.8,no,no,Sometimes,no,no,Sometimes,Public_Transportation,Overweight_Level_II,Yes


## 1. Height as predictor of obesity and overweight

In [3]:
# Select columns
data = df[['Height', 'Obese/Overweight']]
data.head()

,Height,Obese/Overweight
0,1.62,No
1,1.52,No
2,1.80,No
3,1.80,Yes
4,1.78,Yes


In [4]:
# Encode categorical data
label_encoder = LabelEncoder()
data['Obese/Overweight'] = label_encoder.fit_transform(data['Obese/Overweight'])

In [5]:
# Define features (X) and target (y)
X = data[['Height']] 
y = data['Obese/Overweight']   

In [6]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [10]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7152 - loss: 0.6681 - val_accuracy: 0.7544 - val_loss: 0.6260
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - accuracy: 0.7364 - loss: 0.6272 - val_accuracy: 0.7544 - val_loss: 0.5935
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - accuracy: 0.7291 - loss: 0.6053 - val_accuracy: 0.7544 - val_loss: 0.5760
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - accuracy: 0.7229 - loss: 0.5953 - val_accuracy: 0.7544 - val_loss: 0.5698
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - accuracy: 0.7197 - loss: 0.5929 - val_accuracy: 0.7544 - val_loss: 0.5654
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - accuracy: 0.7299 - loss: 0.5852 - val_accuracy: 0.7544 - val_loss: 0.5631
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step - accuracy: 0.7274 - loss: 0.5894 - val_accuracy: 0.7544 - val_loss: 0.5597
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - accuracy: 0.7254 - loss: 0.5835 - val_accura

In [11]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7515 - loss: 0.5547 
Neural Network Accuracy: 0.7352246046066284


## 2. Age as predictor of obesity/overweight

In [12]:
# Select columns
data2 = df[['Age', 'Obese/Overweight']]
data2.head()

,Age,Obese/Overweight
0,21.0,No
1,21.0,No
2,23.0,No
3,27.0,Yes
4,22.0,Yes


In [13]:
# Encode categorical data
label_encoder = LabelEncoder()
data2['Obese/Overweight'] = label_encoder.fit_transform(data2['Obese/Overweight'])

In [14]:
# Define features (X) and target (y)
X = data2[['Age']]
y = data2['Obese/Overweight']   

In [15]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [19]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4404 - loss: 0.7239 - val_accuracy: 0.7515 - val_loss: 0.6689
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - accuracy: 0.7331 - loss: 0.6611 - val_accuracy: 0.7544 - val_loss: 0.6233
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 674us/step - accuracy: 0.7439 - loss: 0.6189 - val_accuracy: 0.7544 - val_loss: 0.5806
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7478 - loss: 0.5764 - val_accuracy: 0.7544 - val_loss: 0.5430
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - accuracy: 0.7439 - loss: 0.5427 - val_accuracy: 0.7544 - val_loss: 0.5177
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - accuracy: 0.7131 - loss: 0.5409 - val_accuracy: 0.7544 - val_loss: 0.5013
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 712us/step - accuracy: 0.7443 - loss: 0.5006 - val_accuracy: 0.7604 - val_loss: 0.4943
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step - accuracy: 0.7302 - loss: 0.5075 - val_accuracy

In [20]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7886 - loss: 0.4365 
Neural Network Accuracy: 0.758865237236023


## 3. Meals per day as predictor of obesity/overweight

In [21]:
# Select columns
data3 = df[['Meals_Per_Day', 'Obese/Overweight']]
data3.head()

,Meals_Per_Day,Obese/Overweight
0,3.0,No
1,3.0,No
2,3.0,No
3,3.0,Yes
4,1.0,Yes


In [22]:
# Encode categorical data
label_encoder = LabelEncoder()
data3['Obese/Overweight'] = label_encoder.fit_transform(data3['Obese/Overweight'])

In [23]:
# Define features (X) and target (y)
X = data3[['Meals_Per_Day']]
y = data3['Obese/Overweight']   

In [24]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [28]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7354 - loss: 0.6613 - val_accuracy: 0.7544 - val_loss: 0.6344
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - accuracy: 0.7270 - loss: 0.6318 - val_accuracy: 0.7544 - val_loss: 0.6140
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 712us/step - accuracy: 0.7285 - loss: 0.6149 - val_accuracy: 0.7544 - val_loss: 0.5938
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - accuracy: 0.7241 - loss: 0.6013 - val_accuracy: 0.7544 - val_loss: 0.5760
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - accuracy: 0.7380 - loss: 0.5815 - val_accuracy: 0.7544 - val_loss: 0.5653
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - accuracy: 0.7261 - loss: 0.5800 - val_accuracy: 0.7544 - val_loss: 0.5559
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - accuracy: 0.7448 - loss: 0.5524 - val_accuracy: 0.7604 - val_loss: 0.5484
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - accuracy: 0.7610 - loss: 0.5478 - val_accura

In [29]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7986 - loss: 0.4890 
Neural Network Accuracy: 0.7777777910232544


## 4. Frequency of physical activity as predictor of obesity/overweight

In [31]:
# Select columns
data4 = df[['Frequency_Physical_Activity', 'Obese/Overweight']]
data4.head()

,Frequency_Physical_Activity,Obese/Overweight
0,0.0,No
1,3.0,No
2,2.0,No
3,2.0,Yes
4,0.0,Yes


In [32]:
# Encode categorical data
label_encoder = LabelEncoder()
data4['Obese/Overweight'] = label_encoder.fit_transform(data4['Obese/Overweight'])

In [33]:
# Define features (X) and target (y)
X = data4[['Frequency_Physical_Activity']]
y = data4['Obese/Overweight']   

In [34]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [35]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [36]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [38]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7315 - loss: 0.6439 - val_accuracy: 0.7544 - val_loss: 0.6024
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - accuracy: 0.7314 - loss: 0.6097 - val_accuracy: 0.7544 - val_loss: 0.5751
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 668us/step - accuracy: 0.7288 - loss: 0.5999 - val_accuracy: 0.7544 - val_loss: 0.5645
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 795us/step - accuracy: 0.7449 - loss: 0.5687 - val_accuracy: 0.7544 - val_loss: 0.5589
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - accuracy: 0.7304 - loss: 0.5828 - val_accuracy: 0.7544 - val_loss: 0.5560
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - accuracy: 0.7274 - loss: 0.5837 - val_accuracy: 0.7544 - val_loss: 0.5526
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - accuracy: 0.7415 - loss: 0.5601 - val_accuracy: 0.7544 - val_loss: 0.5503
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - accuracy: 0.7296 - loss: 0.5803 - val_accura

In [39]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7276 - loss: 0.5402 
Neural Network Accuracy: 0.7210401892662048


## 5. Water intake as predictor of obesity/overweight

In [40]:
# Select columns
data5 = df[['Water_Intake', 'Obese/Overweight']]
data5.head()

,Water_Intake,Obese/Overweight
0,2.0,No
1,3.0,No
2,2.0,No
3,2.0,Yes
4,2.0,Yes


In [41]:
# Encode categorical data
label_encoder = LabelEncoder()
data5['Obese/Overweight'] = label_encoder.fit_transform(data5['Obese/Overweight'])

In [42]:
# Define features (X) and target (y)
X = data5[['Water_Intake']]
y = data5['Obese/Overweight']   

In [43]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [44]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [45]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [46]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [47]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6770 - loss: 0.6675 - val_accuracy: 0.7544 - val_loss: 0.6056
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step - accuracy: 0.7230 - loss: 0.6148 - val_accuracy: 0.7544 - val_loss: 0.5682
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - accuracy: 0.7447 - loss: 0.5774 - val_accuracy: 0.7544 - val_loss: 0.5512
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - accuracy: 0.7346 - loss: 0.5723 - val_accuracy: 0.7544 - val_loss: 0.5482
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 659us/step - accuracy: 0.7499 - loss: 0.5648 - val_accuracy: 0.7544 - val_loss: 0.5454
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step - accuracy: 0.7523 - loss: 0.5546 - val_accuracy: 0.7544 - val_loss: 0.5466
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - accuracy: 0.7102 - loss: 0.5958 - val_accuracy: 0.7544 - val_loss: 0.5450
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step - accuracy: 0.7280 - loss: 0.5836 - val_accura

In [48]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7515 - loss: 0.5015 
Neural Network Accuracy: 0.7352246046066284


## 6. Vegetable intake as predictor of obesity/overweight

In [49]:
# Select columns
data6 = df[['Vegetable_Intake', 'Obese/Overweight']]
data6.head()

,Vegetable_Intake,Obese/Overweight
0,2.0,No
1,3.0,No
2,2.0,No
3,3.0,Yes
4,2.0,Yes


In [50]:
# Encode categorical data
label_encoder = LabelEncoder()
data6['Obese/Overweight'] = label_encoder.fit_transform(data6['Obese/Overweight'])

In [51]:
# Define features (X) and target (y)
X = data6[['Vegetable_Intake']]
y = data6['Obese/Overweight']   

In [52]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [53]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [54]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [55]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [56]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2918 - loss: 0.7275 - val_accuracy: 0.7189 - val_loss: 0.6809
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step - accuracy: 0.7207 - loss: 0.6664 - val_accuracy: 0.7544 - val_loss: 0.6145
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - accuracy: 0.7106 - loss: 0.6270 - val_accuracy: 0.7544 - val_loss: 0.5846
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 674us/step - accuracy: 0.7213 - loss: 0.6112 - val_accuracy: 0.7544 - val_loss: 0.5725
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 677us/step - accuracy: 0.7524 - loss: 0.5746 - val_accuracy: 0.7544 - val_loss: 0.5685
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7163 - loss: 0.6013 - val_accuracy: 0.7544 - val_loss: 0.5667
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step - accuracy: 0.7416 - loss: 0.5782 - val_accuracy: 0.7544 - val_loss: 0.5635
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step - accuracy: 0.7326 - loss: 0.5829 - val_accuracy

In [57]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7515 - loss: 0.5618
Neural Network Accuracy: 0.7352246046066284


## 7. Technology use as predictor of obesity/overweight

In [59]:
# Select columns
data7 = df[['Technology_Use', 'Obese/Overweight']]
data7.head()

,Technology_Use,Obese/Overweight
0,1.0,No
1,0.0,No
2,1.0,No
3,0.0,Yes
4,0.0,Yes


In [60]:
# Encode categorical data
label_encoder = LabelEncoder()
data7['Obese/Overweight'] = label_encoder.fit_transform(data7['Obese/Overweight'])

In [61]:
# Define features (X) and target (y)
X = data7[['Technology_Use']]
y = data7['Obese/Overweight']   

In [62]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [63]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [64]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 20, activation = "relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 10, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 20)             │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 261 (1.02 KB)

 Trainable params: 261 (1.02 KB)

 Non-trainable params: 0 (0.00 B)

In [65]:
# Compile the neural network
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [66]:
# Train the neural network
nn.fit(X_train_scaled, y_train, epochs = 100, verbose = 1, validation_split = 0.2)

Epoch 1/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6306 - loss: 0.6914 - val_accuracy: 0.7278 - val_loss: 0.6526
Epoch 2/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step - accuracy: 0.7356 - loss: 0.6416 - val_accuracy: 0.7544 - val_loss: 0.6076
Epoch 3/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - accuracy: 0.7334 - loss: 0.6062 - val_accuracy: 0.7544 - val_loss: 0.5803
Epoch 4/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.7241 - loss: 0.5923 - val_accuracy: 0.7544 - val_loss: 0.5704
Epoch 5/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - accuracy: 0.7242 - loss: 0.5905 - val_accuracy: 0.7544 - val_loss: 0.5654
Epoch 6/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - accuracy: 0.7210 - loss: 0.5812 - val_accuracy: 0.7544 - val_loss: 0.5629
Epoch 7/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 677us/step - accuracy: 0.7470 - loss: 0.5584 - val_accuracy: 0.7544 - val_loss: 0.5601
Epoch 8/100
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - accuracy: 0.7218 - loss: 0.5746 - val_accura

In [67]:
# Evaluate the neural network
nn_loss, nn_accuracy = nn.evaluate(X_test_scaled, y_test)
print(f"Neural Network Accuracy: {nn_accuracy}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7741 - loss: 0.5571 
Neural Network Accuracy: 0.7494089603424072
